In [22]:
import os
import pandas as pd
from IPython.display import display, Markdown
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate

In [23]:
data_file = 'data/week4_early.csv'

In [24]:
api_key = os.environ.get("OPENAI_API_KEY")

In [25]:
def load_csv(input_csv):
  df = pd.read_csv(input_csv)
  return df

In [26]:
def generate_response(csv_file, query):
    llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)
    df = load_csv(csv_file)
    agent_kwargs = {"handle_parsing_errors":True, 'prefix': f'You are friendly Sports Analytics assistant, an expert in Sports and a world-class Python programmer. You are tasked with creating the best outcomes for your user.'}
    agent = create_pandas_dataframe_agent(llm, df, verbose=True, agent_type=AgentType.OPENAI_FUNCTIONS, agent_kwargs=agent_kwargs)
    response = agent.run(query)
    return (response)

In [27]:
query_text = """
Using df, perform a sports analytics task. The dataset contains columns: `player_name`, `player_position_id`, `rank_ecr`, `projected_points`, and `salary`.

1. **Data Understanding**: Load the df and display the first 5 rows.
2. **Value Metric**: Calculate a new column named 'value' for each player by dividing their `projected_points` by their `salary`.
3. **Player Sorting**: Sort players in descending order based on the 'value' metric.
4. **Initial Lineup Creation**: Assemble an initial team lineup with:
  - One 'QB'
  - Two 'RBs'
  - Three 'WRs'
  - One 'TE'
  - One FLEX (from 'RB', 'WR', or 'TE' but not already in the lineup)
  - One 'DST'
5. **Budget Check**: Calculate the cumulative salary of the lineup, ensuring it's below $50,000.
6. **Optimization**: If the salary exceeds $50,000, replace the player with the lowest 'value' metric with a cheaper, higher 'value' player of the same position. Repeat until under budget.
7. If there is more than 1000 in salary remaining, look to maximize the points
8. **Output**

IMPORTANT: Every <Thought:> must either come with an <Action: and Action Input:> or <Final Answer:>

**Example Output**:
```
Optimal Lineup:
QB: 
RB1: 
RB2:
WR1:
WR2:
WR3:
TE:
FLEX:
DST:
Total Points:
Total Salary:
```

"""



In [28]:
generate_response(data_file, query_text)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.head()'}`


         player_name player_position_id  rank_ecr pos_rank  projected_points  \
0          joshallen                 QB         1      QB1              22.6   
1         jalenhurts                 QB         2      QB2              22.1   
2      justinherbert                 QB         4      QB4              20.8   
3      tuatagovailoa                 QB         5      QB5              20.2   
4  anthonyrichardson                 QB         6      QB6              19.7   

   salary  
0  8200.0  
1  8000.0  
2  7800.0  
3  7400.0  
4  6700.0  
Invoking: `python_repl_ast` with `{'query': "df['value'] = df['projected_points'] / df['salary']\ndf.head()"}`


         player_name player_position_id  rank_ecr pos_rank  projected_points  \
0          joshallen                 QB         1      QB1              22.6   
1         jalenhurts                 QB         2      QB2              2

"<Final Answer:>\n\nOptimal Lineup:\nQB: Russell Wilson\nRB1: Kyren Williams\nRB2: D'Andre Swift\nWR1: Adam Thielen\nWR2: Joshua Palmer\nWR3: Tank Dell\nTE: Pat Freiermuth\nFLEX: Adam Thielen\nDST: Browns\nTotal Points: 102.6\nTotal Salary: 36800.0"